<a href="https://colab.research.google.com/github/zaneprice5/reddit_sentiment/blob/main/reddit_crawler_stock_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Want to create a program that can scrape reddit on demand, pass into sentiment scorer, and rank stocks by their sentiment. 

### I will pass this file along to a github repo from time to time for public access for NLP analysis or any other type of analysis. 

Step 1:
Reddit scraper

list of stock tickers to collect

search query for these tickers on reddit subreddits

append these posts to df

source: https://www.kaggle.com/code/nikhilkhetan/utility-data-scraping-twitter-reddit

In [ ]:
!pip install praw

In [ ]:
import praw
import pandas as pd
import datetime as dt
from tqdm import tqdm
import time
import regex as re
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import spacy
import plotly.express as px

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

In [ ]:
#fill in the below Authentication details from Reddit
def reddit_connection():
    personal_use_script = "SCRIPT"
    client_secret = "CLIENT_SECRET_KEY"
    user_agent = "AGENT_NAME"
    username = "USERNAME"
    password = "PASSWORD"

    reddit = praw.Reddit(client_id=personal_use_script, \
                         client_secret=client_secret, \
                         user_agent=user_agent, \
                         username=username, \
                         password='')
    return reddit

In [ ]:
def scraper(reddit, subreddit, post_limit, comments_limit):

    # Collect reddit comments
    subreddit = reddit.subreddit(subreddit)
    # Collect reddit posts
    new_subreddit = subreddit.new(limit=post_limit)

    topics_dict = { "name":[],
                   "title":[],
                "score":[],
                "id":[], "url":[],
                "comms_num": [],
                "created": [],
                "body":[]}

    print(f"retreive new reddit posts ...")
    for submission in tqdm(new_subreddit):
        topics_dict["name"].append(submission.name)
        topics_dict["title"].append(submission.title)
        topics_dict["score"].append(submission.score)
        topics_dict["id"].append(submission.id)
        topics_dict["url"].append(submission.url)
        topics_dict["comms_num"].append(submission.num_comments)
        topics_dict["created"].append(submission.created)
        topics_dict["body"].append(submission.selftext)

    for comment in tqdm(subreddit.comments(limit = comments_limit)):
        topics_dict["name"].append(comment.name)
        topics_dict["title"].append("Comment")
        topics_dict["score"].append(comment.score)
        topics_dict["id"].append(comment.id)
        topics_dict["url"].append("")
        topics_dict["comms_num"].append(0)
        topics_dict["created"].append(comment.created)
        topics_dict["body"].append(comment.body)

    topics_df = pd.DataFrame(topics_dict)
    print(f"new reddit posts retrieved: {len(topics_df)}")
    topics_df['timestamp'] = topics_df['created'].apply(lambda x: get_date(x))

    return topics_df

In [ ]:
#this function will check for duplicates and not write old posts to the file that are already there.

#if you want to just do an analysis on
def update_and_save_dataset(topics_df, file_path):  
      topics_old_df = pd.read_csv(file_path)
      print(f"past reddit posts: {topics_old_df.shape}")

      topics_all_df = pd.concat([topics_old_df, topics_df], axis=0)
      print(f"new reddit posts: {topics_df.shape[0]} past posts: {topics_old_df.shape[0]} all posts: {topics_all_df.shape[0]}")

      topics_new_df = topics_all_df.drop_duplicates(subset = ["id"], 
                                                    keep='last',     #when dropping duplicates, keep the most recent?
                                                    inplace=False) #doesn't alter original df, just the new one we just assigned.

      topics_new_df.to_csv(file_path, index=False)
      print(f"all reddit posts now in file: {topics_new_df.shape}")

In [ ]:
subreddit = 'wallstreetbets+finance+stocks+investing+financialplanning+personalfinance+retirement+FinanceNews+StockMarket+Superstonk+dividends+FinancialPlanning+StonkFeed+ValueInvesting'
post_limit = 10000
comments_limit = 200000

reddit = reddit_connection()
topics_data_df = scraper(reddit, subreddit, post_limit, comments_limit)

In [ ]:
file_path = '/content/reddit_data.csv'
update_and_save_dataset(topics_data_df, file_path)

past reddit posts: (16820, 9)
new reddit posts: 21866 past posts: 16820 all posts: 38686
all reddit posts now in file: (28266, 9)


In [ ]:
df = pd.read_csv('reddit_data.csv') #read csv into df

# let's drop the rows that don't have any values in 'body' or 'title' because we're here for the text, folks. 
no_na = df[(df['body'].notna()) & (df['title'].notna())]

#I will also filter out by date here because for stock analysis we want very timely data (this is optional)
today = no_na[no_na['timestamp'].str.contains('2023-02-10')]

#filter out the columns we actually want...and for sentiment that's just the body of text. 
text = today[['title', 'body']]

#make sure everything is str dtype
text['title']. astype(str)
text['body'].astype(str)
text

,title,body
6400,Finance for out of state move with elderly mother,"Hi All,\n\nI’m planning on relocating this yea..."
6402,"Observation, not recommending options use…but ...",Just an observation and looking for any wrinkl...
6404,Investing early,Hey everyone just started working/college abou...
6405,Cash reserves for a condo trust: how much is n...,I've been trying to find guidance in this and ...
6407,Is my job under paying me,"I was scheduled for 25 hours last week,but clo..."
...,...,...
22565,Comment,yah its kindof a hiden feature but you can mak...
22566,Comment,Galore? Market sentiment is at extreme greed. ...
22567,Comment,rotate pic.
22568,Comment,I find their recommendations to be pretty deta...


In [ ]:
# print(text['body'].isnull().sum())
# text = text.dropna(subset=['body'])
text


,title,body
6400,Finance for out of state move with elderly mother,"Hi All,\n\nI’m planning on relocating this yea..."
6402,"Observation, not recommending options use…but ...",Just an observation and looking for any wrinkl...
6404,Investing early,Hey everyone just started working/college abou...
6405,Cash reserves for a condo trust: how much is n...,I've been trying to find guidance in this and ...
6407,Is my job under paying me,"I was scheduled for 25 hours last week,but clo..."
...,...,...
22565,Comment,yah its kindof a hiden feature but you can mak...
22566,Comment,Galore? Market sentiment is at extreme greed. ...
22567,Comment,rotate pic.
22568,Comment,I find their recommendations to be pretty deta...


In [ ]:
#filter out the garbage

#Remove handlers
text.title = text.title.apply(lambda x:re.sub('@[^\s]+','',x))
text.body  = text.body.apply(lambda x:re.sub('@[^\s]+','',x))

# Remove URLS
text.title = text.title.apply(lambda x:re.sub(r"http\S+", "", x))
text.body   = text.body.apply(lambda x:re.sub(r"http\S+", "", x))

# Remove all the special characters
text.title = text.title.apply(lambda x:' '.join(re.findall(r'\w+', x)))
text.body   = text.body.apply(lambda x:' '.join(re.findall(r'\w+', x)))

#remove all single characters
text.title = text.title.apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))
text.body   = text.body.apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))

# Substituting multiple spaces with single space
text.title = text.title.apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))
text.body   = text.body.apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))

In [ ]:
text

,title,body
6400,Finance for out of state move with elderly mother,"Hi All,\n\nI’m planning on relocating this yea..."
6402,"Observation, not recommending options use…but ...",Just an observation and looking for any wrinkl...
6404,Investing early,Hey everyone just started working/college abou...
6405,Cash reserves for a condo trust: how much is n...,I've been trying to find guidance in this and ...
6407,Is my job under paying me,"I was scheduled for 25 hours last week,but clo..."
...,...,...
22565,Comment,yah its kindof a hiden feature but you can mak...
22566,Comment,Galore? Market sentiment is at extreme greed. ...
22567,Comment,rotate pic.
22568,Comment,I find their recommendations to be pretty deta...


#### We can do some EDA on the text data that we have collected

In [ ]:
#create an iterable with one sentence per item
# nltk.download('punkt')

# sentence_list = []

# for text1, text2 in zip(text['title'], df['body']):
#     sentences1 = nltk.sent_tokenize(text1)
#     sentences2 = nltk.sent_tokenize(text2)
#     if text1 != 'Comment':
#       sentence_list.extend(sentences1 + sentences2)


# sentence_list

In [ ]:
#keep getting an error, so let's debug

# for i, val in enumerate(text['title']):
#     try:
#         sent_tokenize(val)
#     except Exception as e:
#         print("Error at index", i, ":", val)


In [ ]:
# Iterate through with Spacy and tokenize by the sentence. NLTK keeps throwing an error and I've given up on its tokenization...

nlp = spacy.load("en_core_web_sm")

sentences = []
# for i in range(len(text)):
#   print(i)
  # if i in text.index:
  #   print(i)
for i, row in text.iterrows():
  if text['title'][i] != 'Comment':
    texts = text['title'][i] + " " + text['body'][i]
    # print(texts)
  else:
    texts = text['body'][i]
    doc = nlp(texts)
  for sent in doc.sents:
    # print(doc.sents)
    sentences.append(sent.text)

In [ ]:
len(sentences)

18554

## Step 2: 

Run these posts through sentiment analysis...

there are tons of options for this. 

probably want some type of actual score that can be ranked. 

Rather than generic classes like positive, neutral, negative. 



So now we have a pandas dataframe of our comments/posts from Reddit. 
However, we are only interested in the comments/posts that contain some type of mention of stocks!

So we will make a list of all stock tickers, and then filter our comments/posts df on this list. 

In [ ]:
#read in csv of stock tickers
tickers_df = pd.read_csv('2/file path.csv')
tickers_df['Symbol'].astype(str) 

#no nan
no_emp = tickers_df[tickers_df['Symbol'].notna()]

ticker_list = no_emp.Symbol.values.tolist()
#len(ticker_list)

In [ ]:
#add regex to all of the tickers so that we filter out the nonsense.
new_ticker_list = ['\\b'+ i + '\\b' for i in ticker_list]
# new_ticker_list

In [ ]:
#test list
# search = ['\\s*TSLA*\\s','\\s*AAPL*\\s' ]

####we could search the dataframe for the particular tickers. But, then it will return whole comments and/or posts. We really just want sentences that contain the stock ticker of interest. 
#####therefore, we will search the list for sentences that contain the list of substring tickers.

In [ ]:
def filter_list(text_list, substrings):
    matches = {pattern: [] for pattern in substrings}
    for element in text_list:
        for pattern in substrings:
            if re.search(pattern, element):
                matches[pattern].append(element)
    max_len = max(len(val) for val in matches.values())
    for val in matches.values():
        val += [None] * (max_len - len(val))
    return matches

text_list = sentences
substrings = new_ticker_list[2000:3000]

filtered_list = filter_list(text_list, substrings)
df_matches = pd.DataFrame(filtered_list)
# df_matches.info()
df_matches.head()



# substrings = new_ticker_list[500:1000]
# strings = sentences
# filtered_sentences = [s for s in strings if any(re.match(substring, s) for substring in substrings)]
# print(filtered_sentences)



,\bCZR\b,\bCZWI\b,\bD\b,\bDAC\b,\bDADA\b,\bDAIO\b,\bDAKT\b,\bDAL\b,\bDALN\b,\bDALS\b,...,\bGBNH\b,\bGBNY\b,\bGBR\b,\bGBRGR\b,\bGBRGW\b,\bGBTG\b,\bGBX\b,\bGCBC\b,\bGCI\b,\bGCMG\b
0,None,None,D) OP currently has his money under a mattress...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,None,None,*You have 11584 shares in 1 CS Accounts logged...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,None,None,**You have 458 shares logged from the followin...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,None,None,<|\n|^(score = weighted metric of pubilc profi...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,None,None,<|\n|^(score = weighted metric of pubilc profi...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
# #Now we will filter our DF of comments/posts on this ticker list. 
# ticker_only = text[text['body'].str.contains('|'.join(new_ticker_list[:10]))] 

# #need to work on this regex to contain ONLY the pattern *pattern*
# ticker_only[:200]

Now we have a dataframe with the sentences that have the tickers. 
We need to get the subset of the dataframe with n tickers that have the most corresponding mentions. 

In [ ]:
#n is number of tickers we want to return. Could be highest 5 or highest 50...
n = 5

df_subset = df_matches.loc[:, df_matches.count().nlargest(n).index]
df_subset

,\bDRS\b,\bDD\b,\bF\b,\bE\b,\bEDBL\b
0,Why GME What is DRS Low karma apes feed the bo...,**Total Comments**|894|**Previous Best DD**|\n,F-series have been the best selling vehicles i...,"E.g. If a company pays a .5% dividend, the sto...",Yeah but looks like $EDBL is starting a short ...
1,Why GME What is DRS Low karma apes feed the bo...,**Total Comments**|13|**Previous Best DD**|\n,I would guess the pickup driver tends to fare ...,Imagine FOMO buying TSLA and NVDA at 50x FORWA...,Any thoughts on $EDBL?
2,Why GME What is DRS Low karma apes feed the bo...,See: The entirety of GME DD,I just feel like more than half the time just ...,* P/E is 6.5\n* P/FCF 13.62\n* PEG 0.45\n* Deb...,$EDBL jumped up to first once they reported
3,Why GME What is DRS Low karma apes feed the bo...,In WSB**|2 years ago\n**Total Comments**|991|*...,*You have 11584 shares in 1 CS Accounts logged...,W E T H E B E S T,but I think $EDBL is a relatively short term p...
4,Why GME What is DRS Low karma apes feed the bo...,**Total Comments**|0|**Previous Best DD**|\n,Wtf happened to F?,"35,100\n|6,801,164 |State Street Corp|🔼 53,11...","Yeah there have been a ton of squeezes lately,..."
...,...,...,...,...,...
325,[What is DRS?](https://www.reddit.com/r/Supers...,None,None,None,None
326,That A LOT of extra DRS,None,None,None,None
327,"When I don’t buy direct from Computershare, I ...",None,None,None,None
328,That's why I like the DRS movement.,None,None,None,None


### Step 3: 

run sentiment on these stocks!

In [ ]:
sia = SentimentIntensityAnalyzer()
def sentiment_scores(sentence):
    if sentence is None:
        return None
    return sia.polarity_scores(sentence)["compound"]

for column in df_subset.columns:
  if df_subset[column].dtype == object:
    df_subset[f"{column}_sentiment_scores"] = df_subset[column].apply(sentiment_scores)


In [ ]:
df_subset

,\bDRS\b,\bDD\b,\bF\b,\bE\b,\bEDBL\b,\bDRS\b_sentiment_scores,\bDD\b_sentiment_scores,\bF\b_sentiment_scores,\bE\b_sentiment_scores,\bEDBL\b_sentiment_scores
0,Why GME What is DRS Low karma apes feed the bo...,**Total Comments**|894|**Previous Best DD**|\n,F-series have been the best selling vehicles i...,"E.g. If a company pays a .5% dividend, the sto...",Yeah but looks like $EDBL is starting a short ...,0.8192,0.6369,0.6369,-0.2732,0.5927
1,Why GME What is DRS Low karma apes feed the bo...,**Total Comments**|13|**Previous Best DD**|\n,I would guess the pickup driver tends to fare ...,Imagine FOMO buying TSLA and NVDA at 50x FORWA...,Any thoughts on $EDBL?,0.8192,0.6369,0.2023,0.0000,0.0000
2,Why GME What is DRS Low karma apes feed the bo...,See: The entirety of GME DD,I just feel like more than half the time just ...,* P/E is 6.5\n* P/FCF 13.62\n* PEG 0.45\n* Deb...,$EDBL jumped up to first once they reported,0.8192,0.0000,0.7003,0.4767,0.0000
3,Why GME What is DRS Low karma apes feed the bo...,In WSB**|2 years ago\n**Total Comments**|991|*...,*You have 11584 shares in 1 CS Accounts logged...,W E T H E B E S T,but I think $EDBL is a relatively short term p...,0.8192,0.6369,0.2960,0.0000,-0.3291
4,Why GME What is DRS Low karma apes feed the bo...,**Total Comments**|0|**Previous Best DD**|\n,Wtf happened to F?,"35,100\n|6,801,164 |State Street Corp|🔼 53,11...","Yeah there have been a ton of squeezes lately,...",0.8192,0.6369,-0.5859,0.0000,0.8757
...,...,...,...,...,...,...,...,...,...,...
325,[What is DRS?](https://www.reddit.com/r/Supers...,None,None,None,None,-0.2732,NaN,NaN,NaN,NaN
326,That A LOT of extra DRS,None,None,None,None,0.0000,NaN,NaN,NaN,NaN
327,"When I don’t buy direct from Computershare, I ...",None,None,None,None,0.0000,NaN,NaN,NaN,NaN
328,That's why I like the DRS movement.,None,None,None,None,0.3612,NaN,NaN,NaN,NaN


Now we have columns for the scores of each stock, we can easily run averages/other summary stats that would be of interest to us or client. 